# Tool: Create Power BI Dataset from Table Definitions

In [1]:
import adal
from pypowerbi.client import PowerBIClient
from pypowerbi.dataset import Column, Table, Dataset

In [2]:
# Get the token here
token = {'accessToken': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2FuYWx5c2lzLndpbmRvd3MubmV0L3Bvd2VyYmkvYXBpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNzU3MWE0ODktYmQyOS00ZjM4LWI5YTYtN2M4ODBmOGNkZGYwLyIsImlhdCI6MTY5MzMxNTAyNiwibmJmIjoxNjkzMzE1MDI2LCJleHAiOjE2OTMzMTkxOTksImFjY3QiOjAsImFjciI6IjEiLCJhaW8iOiJBWVFBZS84VUFBQUFFb0phK3hOM2V6ZGNtelBBbGlmZjRBczBrajdUSkhmYlF0aVlmRDVUeE1LZkx6MVdhbDlKNUpRZmtIMlU2WmNxYVZab3p3dmpkbXNxcTV3TGVvWDFzR08vWW9sWklJQnMxOUpUY1B4LzNrb0IycThjUlUvbCtLTDNJc0x0dTNic0haV1BXeExIVElYdlhUUERTY2RRZjhHRE9QNHR5MitRc0tJQmQxOEFRRUk9IiwiYW1yIjpbInB3ZCIsInJzYSJdLCJhcHBpZCI6IjE4ZmJjYTE2LTIyMjQtNDVmNi04NWIwLWY3YmYyYjM5YjNmMyIsImFwcGlkYWNyIjoiMCIsImRldmljZWlkIjoiNmNhNDhmYzgtYmQzNS00YjhhLWIxZjctZmM5ZDA0ZWE1YjJhIiwiZ2l2ZW5fbmFtZSI6IktyaXNobmEgUHJhc2FkIFYuUi4iLCJpcGFkZHIiOiIxMjEuMjQ0LjE5OS4xNzgiLCJuYW1lIjoiS3Jpc2huYSBQcmFzYWQgVi5SLiIsIm9pZCI6ImMxNjM2YTU4LThhODEtNDQ5Yy1hNjQwLTExZTI1YzI1OGZiMCIsIm9ucHJlbV9zaWQiOiJTLTEtNS0yMS0yMTUyOTQ3NjM0LTEwNTc2NDk5Ni0yMTQzNDQ1MjQyLTMzOTQiLCJwdWlkIjoiMTAwMzNGRkY4OTkxMDAxNSIsInJoIjoiMC5BU3NBaWFSeGRTbTlPRS01cG55SUQ0emQ4QWtBQUFBQUFBQUF3QUFBQUFBQUFBQXJBRHMuIiwic2NwIjoiQXBwLlJlYWQuQWxsIENhcGFjaXR5LlJlYWQuQWxsIENhcGFjaXR5LlJlYWRXcml0ZS5BbGwgQ29udGVudC5DcmVhdGUgRGFzaGJvYXJkLlJlYWQuQWxsIERhc2hib2FyZC5SZWFkV3JpdGUuQWxsIERhdGFmbG93LlJlYWQuQWxsIERhdGFmbG93LlJlYWRXcml0ZS5BbGwgRGF0YXNldC5SZWFkLkFsbCBEYXRhc2V0LlJlYWRXcml0ZS5BbGwgR2F0ZXdheS5SZWFkLkFsbCBHYXRld2F5LlJlYWRXcml0ZS5BbGwgUGlwZWxpbmUuRGVwbG95IFBpcGVsaW5lLlJlYWQuQWxsIFBpcGVsaW5lLlJlYWRXcml0ZS5BbGwgUmVwb3J0LlJlYWQuQWxsIFJlcG9ydC5SZWFkV3JpdGUuQWxsIFN0b3JhZ2VBY2NvdW50LlJlYWQuQWxsIFN0b3JhZ2VBY2NvdW50LlJlYWRXcml0ZS5BbGwgVGVuYW50LlJlYWQuQWxsIFRlbmFudC5SZWFkV3JpdGUuQWxsIFVzZXJTdGF0ZS5SZWFkV3JpdGUuQWxsIFdvcmtzcGFjZS5SZWFkLkFsbCBXb3Jrc3BhY2UuUmVhZFdyaXRlLkFsbCIsInNpZ25pbl9zdGF0ZSI6WyJrbXNpIl0sInN1YiI6IjlzNFNucmNCRUFRWm91c0FmUnBfVFVwWmwyM1IwZ0JmYXhQektZMUVsakEiLCJ0aWQiOiI3NTcxYTQ4OS1iZDI5LTRmMzgtYjlhNi03Yzg4MGY4Y2RkZjAiLCJ1bmlxdWVfbmFtZSI6ImtyaXNobmFwcmFzYWQudnJAc29uYXRhLXNvZnR3YXJlLmNvbSIsInVwbiI6ImtyaXNobmFwcmFzYWQudnJAc29uYXRhLXNvZnR3YXJlLmNvbSIsInV0aSI6IklYV0xmYkJWSGstc0hUNms2WEpJQUEiLCJ2ZXIiOiIxLjAiLCJ3aWRzIjpbImI3OWZiZjRkLTNlZjktNDY4OS04MTQzLTc2YjE5NGU4NTUwOSJdfQ.NR5xJ7CKVB9UFRB8hESuewBZPO7LmnfPB-DGA3vG_R6oGaY0eXZEoGjhHhsnA8beRrcFPPrmcX_wHYGl9nHwKkvg89yb57-_6w0WUgL4PZfy9BL19H6imepqUQQUaN29utdVoQvPxHQveByg-p78jOppW4UZr6w3qfCQhPSmB4kFMzsfUBnsG_rqMfAFVWiwvuubkuQmcoRFj7BSYsXCTgGc5oyO2apv5aPzJ0dbzkcDJqYpRNyVA1PW5B-5FywuhkDnDAyu-Jb2NYW9lSbqA2WHKRPXQKZw990dDI33h4DVcSvki5JOH_zEvuijRJrhvMxI_6XqOXzHitfOT0MeHw'}

In [3]:
api_url = 'https://api.powerbi.com'

# create your powerbi api client
client = PowerBIClient(api_url, token)

In [4]:

# create your columns
columns = []
columns.append(Column(name='id', data_type='Int64'))
columns.append(Column(name='name', data_type='string'))
columns.append(Column(name='is_interesting', data_type='boolean'))
columns.append(Column(name='cost_usd', data_type='double'))
columns.append(Column(name='purchase_date', data_type='datetime'))

In [5]:
# create your tables
tables = []
tables.append(Table(name='Products', columns=columns))

In [6]:
# create your dataset
dataset = Dataset(name='Sample Dataset frm Table 2', tables=tables)

In [7]:
# post your dataset!
client.datasets.post_dataset(dataset)

<Dataset {'name': 'Sample Dataset frm Table 2', 'id': '7d763fcf-5de8-4462-8317-1a9ebe5f7d55', 'tables': None, 'add_rows_api_enabled': None, 'configured_by': None, 'is_refreshable': None, 'is_effective_identity_required': None, 'is_effective_identity_roles_required': None, 'is_on_prem_gateway_required': None}>

### Data set is created on the powerbi portal